# Using `rllib` to solve the inventory management custom environment

<img src="images/inv_sim.png" width="750"/>

In [3]:
import gym
from gym.spaces import Box
import numpy as np
from numpy.random import default_rng


class InventoryEnv(gym.Env):
    def __init__(self, config=None):
        """
        Must define self.observation_space and self.action_space here
        """
        
        # Define action space: bounds, space type, shape
        
        # Bound: Shelf space is limited
        self.max_capacity = 4000
        
        # Space type: Better to use Box than Discrete, since Discrete will lead to too many output nodes in the NN
        # Shape: rllib cannot handle scalar actions, so turn it into a numpy array with shape (1,)
        self.action_space = Box(low=np.array([0]), high=np.array([self.max_capacity]))
        
        # Define observation space: bounds, space type, shape
        
        # Shape: The lead time controls the shape of observation space
        self.lead_time = 5
        self.obs_dim = self.lead_time + 4
        
        # Bounds: Define high of the remaining observation space elements
        self.max_mean_daily_demand = 200
        self.max_unit_selling_price = 100
        self.max_daily_holding_cost_per_unit = 5
        
        obs_low = np.zeros((self.obs_dim,))
        obs_high = np.array([self.max_capacity for _ in range(self.lead_time)] +
                            [self.max_mean_daily_demand, self.max_unit_selling_price,
                             self.max_unit_selling_price, self.max_daily_holding_cost_per_unit
                             ]
                            )
        self.observation_space = Box(low=obs_low, high=obs_high)
        
        # The random number generator that will be used throughout the environment
        self.rng = default_rng()
        
        # All instance variables are defined in the __init__() method
        self.current_obs = None
        self.episode_length_in_days = 90
        self.day_num = None

    def reset(self):
        """
        Returns: the observation of the initial state
        Reset the environment to initial state so that a new episode (independent of previous ones) may start
        """
        # Sample parameter values from the parameter space
        
        # Set mean daily demand (lambda)
        mean_daily_demand = self.rng.uniform() * self.max_mean_daily_demand
        
        # Set selling price
        selling_price = self.rng.uniform() * self.max_unit_selling_price
        
        # Set buying price: buying price cannot be higher than selling price
        buying_price = self.rng.uniform() * selling_price
        
        # Set daily holding cose per unit: holding cost cannot be higher than buying_price
        daily_holding_cost_per_unit = self.rng.uniform() * min(buying_price,
                                                               self.max_daily_holding_cost_per_unit
                                                               )
        
        # Return the first observation
        self.current_obs = np.array([0 for _ in range(self.lead_time)] +
                                    [mean_daily_demand, selling_price, buying_price,
                                     daily_holding_cost_per_unit,
                                     ]
                                    )
        self.day_num = 0
        return self.current_obs

    def step(self, action):
        """
        Returns: Given current obs and action, returns the next observation, the reward, done and optionally additional info
        """
        # Action looks like np.array([20.0]). We convert that to float 20.0 for easier calculation
        buys = min(action[0], self.max_capacity - np.sum(self.current_obs[:self.lead_time]))
        
        # Compute next obs
        demand = self.rng.poisson(self.current_obs[self.lead_time])
        next_obs = np.concatenate((self.current_obs[1: self.lead_time],
                                   np.array([buys]),
                                   self.current_obs[self.lead_time:]
                                   )
                                  )
        next_obs[0] += max(0, self.current_obs[0] - demand)
        
        # Compute reward
        reward = (self.current_obs[self.lead_time + 1] * (self.current_obs[0] + self.current_obs[1] - next_obs[0]) -
                  self.current_obs[self.lead_time + 2] * buys - 
                  self.current_obs[self.lead_time + 3] * (next_obs[0] - self.current_obs[1])
                  )
                  
        # Compute done
        self.day_num += 1
        done = False
        if self.day_num >= self.episode_length_in_days:
            done = True
            
        self.current_obs = next_obs

        # info must be a dict
        return self.current_obs, reward, done, {}

    def render(self, mode="human"):
        """
        Returns: None
        Show the current environment state e.g. the graphical window in `CartPole-v1`
        This method must be implemented, but it is OK to have an empty implementation if rendering is not
        important
        """
        pass

    def close(self):
        """
        Returns: None
        This method is optional. Used to cleanup all resources (threads, graphical windows) etc.
        """
        pass
    
    def seed(self, seed=None):
        """
        Returns: List of seeds
        This method is optional. Used to set seeds for the environment's random number generator for 
        obtaining deterministic behavior
        """
        return

In [ ]:
from ray import tune

tune.run("PPO",
         config={"env": InventoryEnv,    # Instead of strings e.g. "CartPole-v1", we pass the custom env class
                 "evaluation_interval": 1000,
                 # Each episode uses different shop params. Need lots of samples to gauge agent's performance
                 "evaluation_num_episodes": 10000    
                 },
         checkpoint_freq=1000,
         local_dir="experiment_vanilla"
         )

(PPOTrainer pid=27788) 2022-10-13 15:14:09,521	INFO trainer.py:2140 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(PPOTrainer pid=27788) 2022-10-13 15:14:09,522	WARNING deprecation.py:45 -- DeprecationWarning: `evaluation_num_episodes` has been deprecated. Use ``evaluation_duration` and `evaluation_duration_unit=episodes`` instead. This will raise an error in the future!
(PPOTrainer pid=27788) 2022-10-13 15:14:09,522	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=27788) 2022-10-13 15:14:09,522	INFO trainer.py:779 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker

Trial name,status,loc
PPO_InventoryEnv_ea493_00000,RUNNING,192.168.0.178:27788


Trial name,status,loc
PPO_InventoryEnv_ea493_00000,RUNNING,192.168.0.178:27788


(PPOTrainer pid=27788) 2022-10-13 15:14:16,755	WARNING deprecation.py:45 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


Trial name,status,loc
PPO_InventoryEnv_ea493_00000,RUNNING,192.168.0.178:27788


...

(PPOTrainer pid=27788) 2022-10-13 15:24:08,568	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 58380.0x the scale of `vf_clip_param`. This means that it will take more than 58380.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_InventoryEnv_ea493_00000,RUNNING,192.168.0.178:27788,89,590.785,356000,-583798,364403,-1.79053e+06,90


Result for PPO_InventoryEnv_ea493_00000:
  agent_timesteps_total: 360000
  custom_metrics: {}
  date: 2022-10-13_15-24-15
  done: false
  episode_len_mean: 90.0
  episode_media: {}
  episode_reward_max: 953603.4691257151
  episode_reward_mean: -597572.9155973935
  episode_reward_min: -1790534.6667692396
  episodes_this_iter: 46
  episodes_total: 4000
  experiment_id: 16c92e0843fe45b6a7fa3194faada972
  hostname: devbox-x299
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 1.708593726158142
          cur_lr: 4.999999873689376e-05
          entropy: 1.2467707395553589
          entropy_coeff: 0.0
          kl: 0.007653938140720129
          model: {}
          policy_loss: -0.002141451695933938
          total_loss: 79705309184.0
          vf_explained_var: 7.646443009434734e-06
          vf_loss: 79705309184.0
    num_agent_steps_sampled: 360000
    num_agent_steps_trained: 360000
    num_steps_sampled: 360000
    num_st

(PPOTrainer pid=27788) 2022-10-13 15:24:15,175	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 59757.0x the scale of `vf_clip_param`. This means that it will take more than 59757.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_InventoryEnv_ea493_00000,RUNNING,192.168.0.178:27788,90,597.344,360000,-597573,953603,-1.79053e+06,90


# My agent does not learn anything!

<img src="images/no_learning.png" width="500" />

## In the next videos in the course, we will learn various tricks that improve performance in a wide range of custom environments.

1. Observation Normalization
2. Action Normalization
3. Reward Scaling
4. Simple Hyperparameter Tuning
5. Advanced hyperparameter tuning e.g. Population Based Training